<a href="https://colab.research.google.com/github/bdi2357/FootballData/blob/main/PlayerPerformanceData/ParesFbref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.system('cp drive/MyDrive/2020_2021_Premier_League_Stats.zip .')

0

In [3]:
!ls

2020_2021_Premier_League_Stats.zip  drive  sample_data


In [7]:
os.path.basename('2020_2021_Premier_League_Stats.zip')

'2020_2021_Premier_League_Stats.zip'

In [42]:
%%time
import pandas as pd
import zipfile
import glob
import os
import re
from datetime import datetime

def extract_date_from_dirname(dirname):
    # Match the pattern of the directory name
    match = re.search(r'-([a-zA-Z]+)-(\d{1,2})-(\d{4})-', dirname)
    if match:
        month_str, day, year = match.groups()
        month = datetime.strptime(month_str, '%B').month
        return datetime(int(year), month, int(day))
    return None

def is_valid_summary_file(filepath):
    try:
        pd.read_csv(filepath)
        return True
    except Exception as e:
        print(e)
        1/0
        return False
"""
def parse_zip_to_stats_table(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")
    extracted_path = os.path.basename(zip_file_path).split(".")[0]
    extracted_files = os.listdir(extracted_path)
    extracted_dir_path = extracted_path #os.path.join(extracted_path, 'PremierLeague_2021-2022')
    extracted_files_in_dir = os.listdir(extracted_dir_path)

    all_data = []

    for team_dir in extracted_files_in_dir:
        #print(team_dir)
        team_dir_path = os.path.join(extracted_dir_path, team_dir)
        for game_dir in os.listdir(team_dir_path):
            game_dir_path = os.path.join(team_dir_path, game_dir)
            game_date = extract_date_from_dirname(game_dir)
            #print(game_date)
            if game_date:
                for summary_file in glob.glob(os.path.join(game_dir_path, '*_Summary.csv')):

                    if is_valid_summary_file(summary_file):

                        summary_df = pd.read_csv(summary_file)
                        summary_df['Game Date'] = game_date
                        summary_df['Team'] = team_dir
                        all_data.append(summary_df)

    if all_data:
        consolidated_df = pd.concat(all_data, ignore_index=True)
        consolidated_df.sort_values(by='Game Date', ascending=False, inplace=True)
        return consolidated_df
    else:
        return pd.DataFrame()
"""
def is_valid_csv_file(filepath):
    try:
        pd.read_csv(filepath)
        return True
    except:
        return False
"""
def parse_zip_to_stats_table(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")

    extracted_path = os.path.basename(zip_file_path).split(".")[0]
    extracted_files = os.listdir(extracted_path)
    extracted_dir_path = extracted_path #os.path.join(extracted_path, 'PremierLeague_2021-2022')
    extracted_files_in_dir = os.listdir(extracted_dir_path)

    valid_postfixes = ["_Miscellaneous Stats", "_Possession", "_Defensive Actions", "_Passing", "_Summary"]

    all_data = []

    for team_dir in extracted_files_in_dir:
        team_dir_path = os.path.join(extracted_dir_path, team_dir)
        for game_dir in os.listdir(team_dir_path):
            game_dir_path = os.path.join(team_dir_path, game_dir)
            game_date = extract_date_from_dirname(game_dir)
            if game_date:
                for csv_file in glob.glob(os.path.join(game_dir_path, '*.csv')):
                    if is_valid_csv_file(csv_file):
                        for postfix in valid_postfixes:
                            if postfix in csv_file:
                                csv_df = pd.read_csv(csv_file)
                                csv_df['Game Date'] = game_date
                                csv_df['Team'] = team_dir
                                csv_df['File Source'] = os.path.basename(csv_file)
                                all_data.append(csv_df)
                                break

    if all_data:
        consolidated_df = pd.concat(all_data, ignore_index=True)
        consolidated_df.sort_values(by='Game Date', ascending=False, inplace=True)
        return consolidated_df
    else:
        return pd.DataFrame()
"""
def extract_team_info(dirname):
    parts = dirname.split('-')
    if len(parts) > 1:
        home_team = parts[0]
        away_team = parts[1]
        return home_team, away_team
    return None, None
def parse_zip_to_stats_table(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")

    extracted_path = os.path.basename(zip_file_path).split(".")[0]
    extracted_files = os.listdir(extracted_path)
    extracted_dir_path = extracted_path #os.path.join(extracted_path, 'PremierLeague_2021-2022')
    extracted_files_in_dir = os.listdir(extracted_dir_path)

    valid_postfixes = ["_Miscellaneous Stats", "_Possession", "_Defensive Actions", "_Passing", "_Summary"]

    all_data = []

    for team_dir in extracted_files_in_dir:
        team_dir_path = os.path.join(extracted_dir_path, team_dir)
        for game_dir in os.listdir(team_dir_path):
            game_dir_path = os.path.join(team_dir_path, game_dir)
            game_date = extract_date_from_dirname(game_dir)
            home_team, away_team = extract_team_info(game_dir)
            if game_date and home_team and away_team:
                game_data = {}
                for csv_file in glob.glob(os.path.join(game_dir_path, '*.csv')):
                    if is_valid_csv_file(csv_file):
                        for postfix in valid_postfixes:
                            if postfix in csv_file:
                                csv_df = pd.read_csv(csv_file)
                                for index, row in csv_df.iterrows():
                                    player_name = row['Player Name']
                                    if player_name not in game_data:
                                        game_data[player_name] = row.to_dict()
                                    else:
                                        game_data[player_name].update(row.to_dict())
                                    game_data[player_name]['Game Date'] = game_date
                                    game_data[player_name]['Team'] = team_dir
                                    game_data[player_name]['Opposing Team'] = away_team if team_dir == home_team else home_team
                                    game_data[player_name]['Home/Away'] = 'Home' if team_dir == home_team else 'Away'
                                break
                for player_stats in game_data.values():
                    all_data.append(player_stats)

    if all_data:
        consolidated_df = pd.DataFrame(all_data)
        consolidated_df.sort_values(by='Game Date', ascending=False, inplace=True)
        return consolidated_df
    else:
        return pd.DataFrame()

# Example usage
zip_file_path = '2020_2021_Premier_League_Stats.zip'
consolidated_df = parse_zip_to_stats_table(zip_file_path)
#import ace_tools as tools; tools.display_dataframe_to_user(name="Consolidated Player Stats", dataframe=consolidated_df)


CPU times: user 1min 24s, sys: 2.61 s, total: 1min 26s
Wall time: 1min 28s


In [43]:
consolidated_df.shape

(26657, 85)

In [23]:
consolidated_df.shape

(26657, 46)

In [50]:
consolidated_df.columns

Index(['Unnamed: 0', 'Player Name', 'Player Number', 'Player', '#', 'Nation',
       'Pos', 'Age', 'Min', 'Att', 'Live', 'Dead', 'FK', 'TB', 'Sw', 'Crs',
       'TI', 'CK', 'In', 'Out', 'Str', 'Cmp', 'Off', 'Blocks', 'Player Key',
       'Player Link', 'Game Date', 'Team', 'Opposing Team', 'Home/Away', 'Gls',
       'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl',
       'Int', 'xG', 'npxG', 'xAG', 'SCA', 'GCA', 'Cmp%', 'PrgP', 'Carries',
       'PrgC', 'Att.1', 'Succ', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd',
       'Att Pen', 'Succ%', 'Tkld', 'Tkld%', 'TotDist', 'PrgDist', '1/3', 'CPA',
       'Mis', 'Dis', 'Rec', 'PrgR', 'TklW', 'Tkl.1', 'Tkl%', 'Lost', 'Pass',
       'Tkl+Int', 'Clr', 'Err', '2CrdY', 'Fls', 'Fld', 'PKwon', 'PKcon', 'OG',
       'Recov', 'Won', 'Won%'],
      dtype='object')

In [58]:
[c for c in consolidated_df.columns if "%" in c]

['Cmp%', 'Succ%', 'Tkld%', 'Tkl%', 'Won%']

In [51]:
consolidated_df["Game Date"].head()

20094   2021-05-29
20100   2021-05-29
21849   2021-05-29
21848   2021-05-29
21847   2021-05-29
Name: Game Date, dtype: datetime64[ns]

In [52]:
consolidated_df.sample(10)

,Unnamed: 0,Player Name,Player Number,Player,#,Nation,Pos,Age,Min,Att,...,Err,2CrdY,Fls,Fld,PKwon,PKcon,OG,Recov,Won,Won%
22122,6,Billy Gilmour,https://fbref.com/en/players/df10e27c/Billy-Gi...,23,23,sct SCO,CM,19-206,27,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,NaN
22329,5,Tommy Doyle,https://fbref.com/en/players/5526deb1/Tommy-Doyle,69,69,eng ENG,MF,19-116,34,NaN,...,NaN,NaN,1.0,0.0,NaN,NaN,0,NaN,NaN,NaN
9719,6,Marc Albrighton,https://fbref.com/en/players/b827d5b3/Marc-Alb...,11,11,eng ENG,RM,31-062,76,29.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0,8.0,2.0,66.7
6536,2,Andriy Yarmolenko,https://fbref.com/en/players/849c826a/Andriy-Y...,7,7,ua UKR,AM,31-065,45,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,1.0,0.0,NaN
26644,1,Marcus Rashford,https://fbref.com/en/players/a1d5bd30/Marcus-R...,10,10,eng ENG,"LW,AM",22-324,90,4.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0,4.0,0.0,0.0
7311,6,Dani Ceballos,https://fbref.com/en/players/c0617e2b/Dani-Ceb...,8,8,es ESP,CM,24-131,66,39.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0,7.0,0.0,NaN
530,5,Jordan Henderson,https://fbref.com/en/players/935e6b8f/Jordan-H...,14,14,eng ENG,"CB,CM",30-179,90,1.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0,13.0,0.0,0.0
24417,12,Raúl Albiol,https://fbref.com/en/players/4bd414c1/Raul-Albiol,3,3,es ESP,CB,35-244,90,45.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0,5.0,3.0,60.0
19684,14,Matt Doherty,https://fbref.com/en/players/d557d734/Matt-Doh...,2,2,ie IRL,RB,28-360,90,NaN,...,NaN,NaN,2.0,1.0,NaN,NaN,0,NaN,NaN,NaN
7121,9,Jonny Evans,https://fbref.com/en/players/f8fcd2a5/Jonny-Evans,6,6,nir NIR,CB,33-013,90,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,3.0,75.0


In [27]:
consolidated_df["Player Name"].head()

20094         N'Golo Kanté
20100    César Azpilicueta
21849        Mateo Kovačić
21848          Mason Mount
21847    Christian Pulisic
Name: Player Name, dtype: object

In [30]:
consolidated_df["Player Number"].head()

20094    https://fbref.com/en/players/b9fbae28/NGolo-Kante
20100    https://fbref.com/en/players/53cad200/Cesar-Az...
21849    https://fbref.com/en/players/79c0821a/Mateo-Ko...
21848    https://fbref.com/en/players/9674002f/Mason-Mount
21847    https://fbref.com/en/players/1bf33a9a/Christia...
Name: Player Number, dtype: object

In [45]:
consolidated_df[consolidated_df["Player Name"] == "N'Golo Kanté"].shape

(83, 85)

In [53]:
consolidated_df[consolidated_df["Player Name"] == "Marcus Rashford"].tail()

,Unnamed: 0,Player Name,Player Number,Player,#,Nation,Pos,Age,Min,Att,...,Err,2CrdY,Fls,Fld,PKwon,PKcon,OG,Recov,Won,Won%
5854,2,Marcus Rashford,https://fbref.com/en/players/a1d5bd30/Marcus-R...,10,10,eng ENG,LW,22-331,90,2.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0,2.0,0.0,NaN
25086,2,Marcus Rashford,https://fbref.com/en/players/a1d5bd30/Marcus-R...,10,10,eng ENG,LW,22-331,90,2.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0,2.0,0.0,NaN
25300,1,Marcus Rashford,https://fbref.com/en/players/a1d5bd30/Marcus-R...,10,10,eng ENG,"FW,MF",22-327,12,NaN,...,NaN,NaN,1.0,0.0,NaN,NaN,0,NaN,NaN,NaN
26644,1,Marcus Rashford,https://fbref.com/en/players/a1d5bd30/Marcus-R...,10,10,eng ENG,"LW,AM",22-324,90,4.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0,4.0,0.0,0.0
5715,1,Marcus Rashford,https://fbref.com/en/players/a1d5bd30/Marcus-R...,10,10,eng ENG,"LW,AM",22-324,90,4.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0,4.0,0.0,0.0


In [54]:
consolidated_df[consolidated_df["Player Name"] == "Marcus Rashford"].tail()[['Game Date', 'Team', 'Opposing Team', 'Home/Away']]

,Game Date,Team,Opposing Team,Home/Away
5854,2020-09-26,Brighton_and_Hove_Albion_Stats,Brighton,Away
25086,2020-09-26,Manchester_United_Stats,Brighton,Away
25300,2020-09-22,Manchester_United_Stats,Luton,Away
26644,2020-09-19,Manchester_United_Stats,Manchester,Away
5715,2020-09-19,Crystal_Palace_Stats,Manchester,Away


In [57]:
 consolidated_df[consolidated_df["Player Name"] == "Marcus Rashford"][['TklW', 'Tkl.1', 'Tkl%', 'Lost', 'Pass','Tkl+Int']]

,TklW,Tkl.1,Tkl%,Lost,Pass,Tkl+Int
25666,2.0,2.0,66.7,1.0,2.0,3.0
25562,0.0,0.0,0.0,1.0,0.0,0.0
255,0.0,0.0,0.0,1.0,0.0,0.0
11257,0.0,1.0,100.0,0.0,1.0,2.0
25484,0.0,1.0,100.0,0.0,1.0,2.0
...,...,...,...,...,...,...
5854,0.0,0.0,0.0,1.0,0.0,0.0
25086,0.0,0.0,0.0,1.0,0.0,0.0
25300,0.0,NaN,NaN,NaN,NaN,NaN
26644,0.0,0.0,0.0,2.0,0.0,0.0


In [47]:
len(consolidated_df["Player Name"].unique())

1948

In [36]:
"abxz" in "ppabxwqqwe"

False

In [60]:
import pandas as pd
import zipfile
import glob
import os
import re
from datetime import datetime

def extract_date_from_dirname(dirname):
    match = re.search(r'-([a-zA-Z]+)-(\d{1,2})-(\d{4})-', dirname)
    if match:
        month_str, day, year = match.groups()
        month = datetime.strptime(month_str, '%B').month
        return datetime(int(year), month, int(day))
    return None

def extract_team_info(dirname):
    parts = dirname.split('-')
    if len(parts) > 1:
        home_team = parts[0]
        away_team = parts[1]
        return home_team, away_team
    return None, None

def clean_team_name(team_dir):
    return team_dir.replace('_Stats', '').replace('_', ' ')

def is_valid_csv_file(filepath):
    try:
        pd.read_csv(filepath)
        return True
    except:
        return False

def compute_percentages(df):
    if 'Cmp' in df.columns and 'Att' in df.columns:
        df['Cmp%'] = df['Cmp'] / df['Att'] * 100
    if 'Tkl' in df.columns and 'Att' in df.columns:
        df['Tkl%'] = df['Tkl'] / df['Att'] * 100
    if 'Succ' in df.columns and 'Att' in df.columns:
        df['Succ%'] = df['Succ'] / df['Att'] * 100
    if 'Tkld' in df.columns and 'Att' in df.columns:
        df['Tkld%'] = df['Tkld'] / df['Att'] * 100

def parse_zip_to_stats_table(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")

    extracted_path = os.path.basename(zip_file_path).split(".")[0]
    extracted_files = os.listdir(extracted_path)
    extracted_dir_path = extracted_path #os.path.join(extracted_path, 'PremierLeague_2021-2022')
    extracted_files_in_dir = os.listdir(extracted_dir_path)

    valid_postfixes = ["_Miscellaneous Stats", "_Possession", "_Defensive Actions", "_Passing", "_Summary"]

    all_data = []

    for team_dir in extracted_files_in_dir:
        team_dir_path = os.path.join(extracted_dir_path, team_dir)
        team_name = clean_team_name(team_dir)
        for game_dir in os.listdir(team_dir_path):
            game_dir_path = os.path.join(team_dir_path, game_dir)
            game_date = extract_date_from_dirname(game_dir)
            home_team, away_team = extract_team_info(game_dir)
            if game_date and home_team and away_team:
                game_data = {}
                for csv_file in glob.glob(os.path.join(game_dir_path, '*.csv')):
                    if is_valid_csv_file(csv_file):
                        for postfix in valid_postfixes:
                            if postfix in csv_file:
                                csv_df = pd.read_csv(csv_file)
                                for index, row in csv_df.iterrows():
                                    player_name = row['Player Name']
                                    if player_name not in game_data:
                                        game_data[player_name] = row.to_dict()
                                    else:
                                        game_data[player_name].update(row.to_dict())
                                    game_data[player_name]['Game Date'] = game_date
                                    game_data[player_name]['Team'] = team_name
                                    game_data[player_name]['Opposing Team'] = away_team if team_name == home_team else home_team
                                    game_data[player_name]['Home/Away'] = 'Home' if team_name == home_team else 'Away'
                                    game_data[player_name]['Game Dir'] = game_dir
                                break
                for player_stats in game_data.values():
                    all_data.append(player_stats)

                # Add a team row that sums values and computes percentages
                team_stats = pd.DataFrame(game_data.values()).sum(numeric_only=True)
                team_stats['Player Name'] = 'Team Total'
                team_stats['Game Date'] = game_date
                team_stats['Team'] = team_name
                team_stats['Opposing Team'] = away_team if team_name == home_team else home_team
                team_stats['Home/Away'] = 'Home' if team_name == home_team else 'Away'
                team_stats['Game Dir'] = game_dir
                compute_percentages(team_stats)
                all_data.append(team_stats.to_dict())

    if all_data:
        consolidated_df = pd.DataFrame(all_data)
        consolidated_df.sort_values(by='Game Date', ascending=False, inplace=True)

        # Compute the result for each team
        def determine_result(row):
            if row['Player Name'] == 'Team Total':
                team_score = row['Gls'] if 'Gls' in row else 0
                # Find the OG of the opposing team
                opposing_team = row['Opposing Team']
                game_dir = row['Game Dir']
                opposing_og = consolidated_df[
                    (consolidated_df['Game Dir'] == game_dir) &
                    (consolidated_df['Team'] == opposing_team) &
                    (consolidated_df['Player Name'] == 'Team Total')
                ]['OG'].sum()
                opponent_score = row['Opp Gls'] if 'Opp Gls' in row else 0
                total_opponent_score = opponent_score + opposing_og
                if team_score > total_opponent_score:
                    return 'Win'
                elif team_score < total_opponent_score:
                    return 'Loss'
                else:
                    return 'Draw'
            return None

        consolidated_df['Result'] = consolidated_df.apply(determine_result, axis=1)
        consolidated_df.drop(columns=['Game Dir'], inplace=True)  # Remove the helper column

        return consolidated_df
    else:
        return pd.DataFrame()

# Example usage
zip_file_path = '2020_2021_Premier_League_Stats.zip'
consolidated_df = parse_zip_to_stats_table(zip_file_path)
#import ace_tools as tools; tools.display_dataframe_to_user(name="Consolidated Player Stats", dataframe=consolidated_df)
#consolidated_df.head()


AttributeError: 'Series' object has no attribute 'columns'

In [62]:
%%time
import pandas as pd
import zipfile
import glob
import os
import re
from datetime import datetime

def extract_date_from_dirname(dirname):
    match = re.search(r'-([a-zA-Z]+)-(\d{1,2})-(\d{4})-', dirname)
    if match:
        month_str, day, year = match.groups()
        month = datetime.strptime(month_str, '%B').month
        return datetime(int(year), month, int(day))
    return None

def extract_team_info(dirname):
    parts = dirname.split('-')
    if len(parts) > 1:
        home_team = parts[0]
        away_team = parts[1]
        return home_team, away_team
    return None, None

def clean_team_name(team_dir):
    return team_dir.replace('_Stats', '').replace('_', ' ')

def is_valid_csv_file(filepath):
    try:
        pd.read_csv(filepath)
        return True
    except:
        return False

def compute_percentages(df):
    if 'Cmp' in df.columns and 'Att' in df.columns:
        df['Cmp%'] = df['Cmp'] / df['Att'] * 100
    if 'Tkl' in df.columns and 'Att' in df.columns:
        df['Tkl%'] = df['Tkl'] / df['Att'] * 100
    if 'Succ' in df.columns and 'Att' in df.columns:
        df['Succ%'] = df['Succ'] / df['Att'] * 100
    if 'Tkld' in df.columns and 'Att' in df.columns:
        df['Tkld%'] = df['Tkld'] / df['Att'] * 100

def parse_zip_to_stats_table(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")

    extracted_path = os.path.basename(zip_file_path).split(".")[0]
    extracted_files = os.listdir(extracted_path)
    extracted_dir_path = extracted_path #os.path.join(extracted_path, 'PremierLeague_2021-2022')
    extracted_files_in_dir = os.listdir(extracted_dir_path)


    valid_postfixes = ["_Miscellaneous Stats", "_Possession", "_Defensive Actions", "_Passing", "_Summary"]

    all_data = []

    for team_dir in extracted_files_in_dir:
        team_dir_path = os.path.join(extracted_dir_path, team_dir)
        team_name = clean_team_name(team_dir)
        for game_dir in os.listdir(team_dir_path):
            game_dir_path = os.path.join(team_dir_path, game_dir)
            game_date = extract_date_from_dirname(game_dir)
            home_team, away_team = extract_team_info(game_dir)
            if game_date and home_team and away_team:
                game_data = {}
                for csv_file in glob.glob(os.path.join(game_dir_path, '*.csv')):
                    if is_valid_csv_file(csv_file):
                        for postfix in valid_postfixes:
                            if postfix in csv_file:
                                csv_df = pd.read_csv(csv_file)
                                for index, row in csv_df.iterrows():
                                    player_name = row['Player Name']
                                    if player_name not in game_data:
                                        game_data[player_name] = row.to_dict()
                                    else:
                                        game_data[player_name].update(row.to_dict())
                                    game_data[player_name]['Game Date'] = game_date
                                    game_data[player_name]['Team'] = team_name
                                    game_data[player_name]['Opposing Team'] = away_team if team_name == home_team else home_team
                                    game_data[player_name]['Home/Away'] = 'Home' if team_name == home_team else 'Away'
                                    game_data[player_name]['Game Dir'] = game_dir
                                break
                for player_stats in game_data.values():
                    all_data.append(player_stats)

                # Add a team row that sums values and computes percentages
                team_stats = pd.DataFrame(game_data.values()).sum(numeric_only=True)
                team_stats['Player Name'] = 'Team Total'
                team_stats['Game Date'] = game_date
                team_stats['Team'] = team_name
                team_stats['Opposing Team'] = away_team if team_name == home_team else home_team
                team_stats['Home/Away'] = 'Home' if team_name == home_team else 'Away'
                team_stats['Game Dir'] = game_dir
                team_stats_df = pd.DataFrame([team_stats])
                compute_percentages(team_stats_df)
                all_data.append(team_stats_df.to_dict(orient='records')[0])

    if all_data:
        consolidated_df = pd.DataFrame(all_data)
        consolidated_df.sort_values(by='Game Date', ascending=False, inplace=True)

        # Compute the result for each team
        def determine_result(row):
            if row['Player Name'] == 'Team Total':
                team_score = row['Gls'] if 'Gls' in row else 0
                # Find the OG of the opposing team
                opposing_team = row['Opposing Team']
                game_dir = row['Game Dir']
                opposing_og = consolidated_df[
                    (consolidated_df['Game Dir'] == game_dir) &
                    (consolidated_df['Team'] == opposing_team) &
                    (consolidated_df['Player Name'] == 'Team Total')
                ]['OG'].sum()
                opponent_score = row['Opp Gls'] if 'Opp Gls' in row else 0
                total_opponent_score = opponent_score + opposing_og
                if team_score > total_opponent_score:
                    return 'Win'
                elif team_score < total_opponent_score:
                    return 'Loss'
                else:
                    return 'Draw'
            return None

        consolidated_df['Result'] = consolidated_df.apply(determine_result, axis=1)
        consolidated_df.drop(columns=['Game Dir'], inplace=True)  # Remove the helper column

        return consolidated_df
    else:
        return pd.DataFrame()

# Example usage
zip_file_path = '2020_2021_Premier_League_Stats.zip'
consolidated_df = parse_zip_to_stats_table(zip_file_path)
# Display the dataframe
consolidated_df.head()


CPU times: user 1min 52s, sys: 2.59 s, total: 1min 55s
Wall time: 1min 57s


,Unnamed: 0,Player Name,Player Number,Player,#,Nation,Pos,Age,Min,Att,...,2CrdY,Fls,Fld,PKwon,PKcon,OG,Recov,Won,Won%,Result
20821,6.0,N'Golo Kanté,https://fbref.com/en/players/b9fbae28/NGolo-Kante,7.0,7.0,fr FRA,CM,30-061,90.0,3.0,...,0.0,0.0,2.0,0.0,0.0,0.0,10.0,4.0,57.1,None
20819,4.0,Kai Havertz,https://fbref.com/en/players/fed7cb61/Kai-Havertz,29.0,29.0,de GER,"AM,FW",21-352,90.0,3.0,...,0.0,1.0,3.0,0.0,0.0,0.0,4.0,2.0,33.3,None
20826,11.0,Andreas Christensen,https://fbref.com/en/players/1cb49278/Andreas-...,4.0,4.0,dk DEN,CB,25-049,52.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,None
20825,10.0,Thiago Silva,https://fbref.com/en/players/86e7deaf/Thiago-S...,6.0,6.0,br BRA,CB,36-249,38.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,100.0,None
20824,9.0,Antonio Rüdiger,https://fbref.com/en/players/18b896d6/Antonio-...,2.0,2.0,de GER,CB,28-087,90.0,3.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,66.7,None


In [63]:
consolidated_df.shape

(27617, 86)

In [64]:
consolidated_df.columns

Index(['Unnamed: 0', 'Player Name', 'Player Number', 'Player', '#', 'Nation',
       'Pos', 'Age', 'Min', 'Att', 'Live', 'Dead', 'FK', 'TB', 'Sw', 'Crs',
       'TI', 'CK', 'In', 'Out', 'Str', 'Cmp', 'Off', 'Blocks', 'Player Key',
       'Player Link', 'Game Date', 'Team', 'Opposing Team', 'Home/Away', 'Gls',
       'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl',
       'Int', 'xG', 'npxG', 'xAG', 'SCA', 'GCA', 'Cmp%', 'PrgP', 'Carries',
       'PrgC', 'Att.1', 'Succ', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd',
       'Att Pen', 'Succ%', 'Tkld', 'Tkld%', 'TotDist', 'PrgDist', '1/3', 'CPA',
       'Mis', 'Dis', 'Rec', 'PrgR', 'TklW', 'Tkl.1', 'Tkl%', 'Lost', 'Pass',
       'Tkl+Int', 'Clr', 'Err', '2CrdY', 'Fls', 'Fld', 'PKwon', 'PKcon', 'OG',
       'Recov', 'Won', 'Won%', 'Result'],
      dtype='object')